In [ ]:
!pip install pyomo
import pyomo.environ as pyo
from pyomo.opt import SolverManagerFactory
import pandas as pd

In [15]:
model = pyo.ConcreteModel()

model.i = pyo.RangeSet(1,3)

D = pd.read_excel('S7P3_Data.xlsx', sheet_name = 'Sheet1', header=0, index_col=0, usecols='A:J', nrows=3)
model.Vt = pyo.Param(initialize = 350)
Vt=model.Vt
model.DPt = pyo.Param(initialize = 400)
DPt=model.DPt
model.Wmax = pyo.Param(initialize = 2950)
Wmax=model.Wmax
model.Npmax = pyo.Param(initialize = 3)
Npmax=model.Npmax
model.Nsmax = pyo.Param(initialize = 3)
Nsmax=model.Nsmax

model.x = pyo.Var(model.i, domain = pyo.NonNegativeReals, bounds = (0,1))
x=model.x

model.v = pyo.Var(model.i, domain = pyo.NonNegativeReals, bounds = (0,Vt))
v=model.v

model.w = pyo.Var(model.i, domain = pyo.NonNegativeReals, bounds = (0,Wmax))
w=model.w

model.DP = pyo.Var(model.i, domain = pyo.NonNegativeReals, bounds = (0,DPt))
DP=model.DP

def Power_bound(model, i):
  return (0, D['Pmax'][i])
model.P = pyo.Var(model.i, domain = pyo.NonNegativeReals, bounds = Power_bound)
P=model.P

model.Np = pyo.Var(model.i, domain = pyo.NonNegativeIntegers, bounds = (0,3))
Np=model.Np

model.Ns = pyo.Var(model.i, domain = pyo.NonNegativeIntegers, bounds = (0,3))
Ns=model.Ns

model.z = pyo.Var(model.i, domain = pyo.Binary)
z=model.z

def objective(model):
  return sum((D['Co'][i] + (D['Cp'][i] * P[i])) * Np[i] * Ns[i] * z[i] for i in model.i)
model.obj = pyo.Objective(rule = objective, sense = pyo.minimize)

def con1(model,i):
  return sum(x[i] for i in model.i) == 1
model.con1 = pyo.Constraint(rule = con1)

def con2(model,i):
  return P[i] - D['alpha'][i] * (w[i]/Wmax)**3 - D['beta'][i] * (w[i]/Wmax)**2 * v[i] - D['gamma'][i] * (w[i]/Wmax) * (v[i]**2) == 0
model.con2 = pyo.Constraint(model.i, rule = con2)

def con3(model,i):
  return  DP[i] - D['a'][i] * (w[i]/Wmax)**2 - D['b'][i] * (w[i]/Wmax) * v[i] -   D['c'][i] * (v[i]**2) == 0
model.con3 = pyo.Constraint(model.i, rule = con3)

def con4(model,i):
  return v[i] * Np[i] - x[i] * Vt == 0
model.con4 = pyo.Constraint(model.i, rule = con4)

def con5(model,i):
  return  DPt * z[i] - DP[i] * Ns[i] == 0
model.con5 = pyo.Constraint(model.i, rule = con5)

def con6(model,i):
  return  P[i] <= z[i] * D['Pmax'][i]
model.con6 = pyo.Constraint(model.i, rule = con6)

def con7(model,i):
  return  DP[i] <= z[i] * DPt
model.con7 = pyo.Constraint(model.i, rule = con7)

def con8(model,i):
  return  v[i] <= z[i] * Vt
model.con8 = pyo.Constraint(model.i, rule = con8)

def con9(model,i):
  return  w[i] <= z[i] * Wmax
model.con9 = pyo.Constraint(model.i, rule = con9)

def con10(model,i):
  return  Np[i] <= z[i] * Npmax
model.con10 = pyo.Constraint(model.i, rule = con10)

def con11(model,i):
  return  Ns[i] <= z[i] * Nsmax
model.con11 = pyo.Constraint(model.i, rule = con11)

def con12(model,i):
  return  x[i] <= z[i]
model.con12 = pyo.Constraint(model.i, rule = con12)

In [ ]:
import os
os.environ['NEOS_EMAIL']=''
solver = SolverManagerFactory('neos') #SolverManagerFactory should be imported instead of SolverFactory
result = solver.solve(model,opt='couenne')
print(result)
print('objective function =', model.obj())
for i in model.i:
  print('Number of Parallel Lines at level',i,'is =',Np[i]())
  print('Number of Pumps at each line at level',i,'is =',Ns[i]())
